In [3]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime 
from datetime import timedelta


# Edoofy app information
edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

# UMS app information
ums_base_url = "https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

# Asynchronous function to fetch data from a table
async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df




async def main():
    
    async with aiohttp.ClientSession() as session:
        #fetch 
        ums_att_summary_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")
        ums_student_df = await fetch_table_data(session, ums_base_url, ums_headers, "Student")
        #fetch only IEs
        student_constraints = [           
            {'key': 'indian-edoofian', 'constraint_type': 'equals', 'value': 'yes'}
        ]
        edoofy_student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student", constraints=student_constraints)  
        
await main()


Fetching Attendance-Summary data from https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj... Cursor: 0
Fetched 30 new records, Total fetched: 30
Exiting loop, fetched less than 100 records.
Fetched 30 records for Attendance-Summary.
Fetching Student data from https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj... Cursor: 0
Fetched 57 new records, Total fetched: 57
Exiting loop, fetched less than 100 records.
Fetched 57 records for Student.
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 300
Fetched 100 new records, Total fetched: 40